<a href="https://colab.research.google.com/github/paolala24/proyecto_final/blob/main/Proyecto_SyS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ── Instalación de librerías para la aplicación ──────────────────────────
!pip install streamlit librosa --quiet
                                     # Streamlit: interfaz web
                                     # Librosa: carga y análisis de audio

# ── Descarga y configuración de Cloudflared ─────────────────────────────
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64 -O cloudflared
                                     # Descarga la última versión del binario
!chmod +x cloudflared                # Asigna permiso de ejecución al archivo
!mv cloudflared /usr/local/bin/cloudflared
                                     # Mueve el binario a una carpeta del PATH
!pip install streamlit scipy soundfile matplotlib pytube pydub ffmpeg-python
!apt-get install -y ffmpeg
!pip install --upgrade pytube

!pip install yt-dlp
!pip install streamlit scipy soundfile matplotlib pydub ffmpeg-python
!apt-get install -y ffmpeg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.3/44.3 kB 1.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.9/9.9 MB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.1/79.1 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.6/57.6 kB 2.4 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 175.4/175.4 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 51.5 MB/s eta 0:00:00
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
ffmpeg is already the newest version (7:4.4.2-0ubuntu0.22.04.1).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [10]:
%%writefile dashboard.py

import streamlit as st
import numpy as np
import matplotlib.pyplot as plt
from scipy.signal import butter, lfilter, freqz
from scipy.signal import hilbert, freqz

# ---- SIDEBAR ----
st.sidebar.title("Anotomia de una Señal Inalambrica")
section = st.sidebar.selectbox(
    "Selecciona el módulo",
    [
        "Dominio de la Frecuencia",
        "Construyendo Señales I/Q",
        "Modulación QAM",
        "Canal AWGN + Demodulación + Constelación"
    ]
)

# ---- SECCIÓN: Dominio de la Frecuencia ----
if section == "Dominio de la Frecuencia":
    st.title("Dominio de la Frecuencia")
    st.markdown("""
    **Simulación básica:** Genera señales sintéticas, aplica FFT, diseña y aplica un filtro paso-bajo y visualiza los resultados en tiempo y frecuencia (incluyendo diagramas de Bode).
    """)

    # --- Parámetros generales ---
    fs = st.sidebar.slider("Frecuencia de muestreo (Hz)", 100, 2000, 500)
    T = st.sidebar.slider("Duración de la señal (segundos)", 0.1, 2.0, 1.0)
    t = np.arange(0, T, 1/fs)

    # --- Selección de señal ---
    signal_type = st.sidebar.selectbox(
        "Tipo de señal",
        ["Seno", "Suma de senos", "Seno + Ruido"]
    )

    if signal_type == "Seno":
        A = st.sidebar.slider("Amplitud", 0.1, 5.0, 1.0)
        f = st.sidebar.slider("Frecuencia (Hz)", 1, int(fs//2)-1, 10)
        x = A * np.sin(2*np.pi*f*t)
        st.write(f"**Señal:** {A}·sin(2π·{f}·t)")
    elif signal_type == "Suma de senos":
        A1 = st.sidebar.slider("Amplitud Seno 1", 0.1, 5.0, 1.0)
        f1 = st.sidebar.slider("Frecuencia Seno 1 (Hz)", 1, int(fs//2)-1, 10)
        A2 = st.sidebar.slider("Amplitud Seno 2", 0.1, 5.0, 0.5)
        f2 = st.sidebar.slider("Frecuencia Seno 2 (Hz)", 1, int(fs//2)-1, 30)
        x = A1 * np.sin(2*np.pi*f1*t) + A2 * np.sin(2*np.pi*f2*t)
        st.write(f"**Señal:** {A1}·sin(2π·{f1}·t) + {A2}·sin(2π·{f2}·t)")
    elif signal_type == "Seno + Ruido":
        A = st.sidebar.slider("Amplitud", 0.1, 5.0, 1.0)
        f = st.sidebar.slider("Frecuencia (Hz)", 1, int(fs//2)-1, 10)
        ruido = st.sidebar.slider("Nivel de ruido", 0.01, 2.0, 0.3)
        x = A * np.sin(2*np.pi*f*t) + ruido * np.random.randn(len(t))
        st.write(f"**Señal:** {A}·sin(2π·{f}·t) + Ruido (σ={ruido})")

    # --- Filtro paso-bajo Butterworth ---
    st.sidebar.markdown("---")
    st.sidebar.markdown("### Filtro paso-bajo Butterworth")
    fc = st.sidebar.slider("Frecuencia de corte (Hz)", 1, int(fs//2)-1, 20)
    orden = st.sidebar.slider("Orden del filtro", 1, 10, 3)

    # Diseño y aplicación del filtro
    b, a = butter(orden, fc/(fs/2), btype='low')
    x_filt = lfilter(b, a, x)

    # --- Visualizaciones ---
    st.subheader("Señal en el tiempo")
    fig1, ax1 = plt.subplots()
    ax1.plot(t, x, label="Original")
    ax1.plot(t, x_filt, label="Filtrada", linestyle='--')
    ax1.set_xlabel("Tiempo (s)")
    ax1.legend()
    st.pyplot(fig1)

    st.subheader("FFT (Espectro de frecuencia)")
    X = np.fft.fft(x)
    X_filt = np.fft.fft(x_filt)
    freqs = np.fft.fftfreq(len(t), 1/fs)
    fig2, ax2 = plt.subplots()
    ax2.plot(freqs[:len(freqs)//2], np.abs(X[:len(freqs)//2]), label="Original")
    ax2.plot(freqs[:len(freqs)//2], np.abs(X_filt[:len(freqs)//2]), label="Filtrada", linestyle='--')
    ax2.set_xlabel("Frecuencia (Hz)")
    ax2.set_ylabel("Magnitud")
    ax2.legend()
    st.pyplot(fig2)

    st.subheader("Diagrama de Bode (Filtro)")
    w, h = freqz(b, a, worN=8000)
    fig3, (ax_mag, ax_phase) = plt.subplots(2, 1, figsize=(7,6))
    ax_mag.plot((fs * 0.5 / np.pi) * w, np.abs(h))
    ax_mag.set_title('Magnitud')
    ax_mag.set_ylabel('|H(f)|')
    ax_mag.set_xlabel('Frecuencia (Hz)')
    ax_mag.grid()
    ax_phase.plot((fs * 0.5 / np.pi) * w, np.angle(h))
    ax_phase.set_title('Fase')
    ax_phase.set_ylabel('Fase (radianes)')
    ax_phase.set_xlabel('Frecuencia (Hz)')
    ax_phase.grid()
    fig3.tight_layout()
    st.pyplot(fig3)

    st.info(
        "Ajusta los parámetros para observar cómo el filtro afecta la señal y su espectro. "
        "El diagrama de Bode muestra la respuesta en frecuencia del filtro paso-bajo utilizado."
    )

# ---- SECCIÓN: Construyendo Señales I/Q ----
elif section == "Construyendo Señales I/Q":
    st.title("Construyendo Señales I/Q")
    st.markdown("""
    Se toma una señal mensaje (I), se aplica la Transformada de Hilbert para generar la señal en cuadratura (Q) y se visualiza el desfase. Se presentan gráficas en el dominio del tiempo y la frecuencia.
    """)

    # Parámetros
    fs = st.sidebar.slider("Frecuencia de muestreo (Hz)", 100, 2000, 500)
    T = st.sidebar.slider("Duración (s)", 0.1, 2.0, 1.0)
    t = np.arange(0, T, 1/fs)

    tipo = st.sidebar.selectbox("Tipo de señal I (mensaje)", ["Seno", "Suma de senos", "Seno + Ruido"])
    if tipo == "Seno":
        A = st.sidebar.slider("Amplitud", 0.1, 5.0, 1.0)
        f = st.sidebar.slider("Frecuencia (Hz)", 1, int(fs//2)-1, 10)
        i_signal = A * np.cos(2 * np.pi * f * t)
        st.write(f"**Señal I:** {A}·cos(2π·{f}·t)")
    elif tipo == "Suma de senos":
        A1 = st.sidebar.slider("Amplitud Seno 1", 0.1, 5.0, 1.0)
        f1 = st.sidebar.slider("Frecuencia Seno 1 (Hz)", 1, int(fs//2)-1, 10)
        A2 = st.sidebar.slider("Amplitud Seno 2", 0.1, 5.0, 0.5)
        f2 = st.sidebar.slider("Frecuencia Seno 2 (Hz)", 1, int(fs//2)-1, 30)
        i_signal = A1 * np.cos(2 * np.pi * f1 * t) + A2 * np.cos(2 * np.pi * f2 * t)
        st.write(f"**Señal I:** {A1}·cos(2π·{f1}·t) + {A2}·cos(2π·{f2}·t)")
    elif tipo == "Seno + Ruido":
        A = st.sidebar.slider("Amplitud", 0.1, 5.0, 1.0)
        f = st.sidebar.slider("Frecuencia (Hz)", 1, int(fs//2)-1, 10)
        ruido = st.sidebar.slider("Nivel de ruido", 0.01, 2.0, 0.3)
        i_signal = A * np.cos(2 * np.pi * f * t) + ruido * np.random.randn(len(t))
        st.write(f"**Señal I:** {A}·cos(2π·{f}·t) + Ruido (σ={ruido})")

    # Señal Q usando Hilbert
    analytic_signal = hilbert(i_signal)
    q_signal = np.imag(analytic_signal)

    # Gráfica tiempo
    st.subheader("Señales I y Q en el tiempo")
    fig1, ax1 = plt.subplots()
    ax1.plot(t, i_signal, label="I (mensaje)")
    ax1.plot(t, q_signal, label="Q (cuadratura)", linestyle='--')
    ax1.set_xlabel("Tiempo (s)")
    ax1.legend()
    st.pyplot(fig1)

    # Diagrama de Lissajous (Q vs I)
    st.subheader("Diagrama de Lissajous (Q vs I)")
    fig2, ax2 = plt.subplots()
    ax2.plot(i_signal, q_signal)
    ax2.set_xlabel("I (mensaje)")
    ax2.set_ylabel("Q (cuadratura)")
    ax2.set_title("Diagrama de Lissajous")
    st.pyplot(fig2)

    # Espectros de ambas señales
    st.subheader("Espectros de I y Q")
    I_fft = np.fft.fft(i_signal)
    Q_fft = np.fft.fft(q_signal)
    freqs = np.fft.fftfreq(len(t), 1/fs)
    fig3, ax3 = plt.subplots()
    ax3.plot(freqs[:len(freqs)//2], np.abs(I_fft[:len(freqs)//2]), label="I")
    ax3.plot(freqs[:len(freqs)//2], np.abs(Q_fft[:len(freqs)//2]), label="Q", linestyle='--')
    ax3.set_xlabel("Frecuencia (Hz)")
    ax3.set_ylabel("Magnitud")
    ax3.legend()
    st.pyplot(fig3)

    # Diagrama de Bode (filtro Hilbert)
    st.subheader("Diagrama de Bode (Transformada de Hilbert)")
    # El filtro de Hilbert ideal en frecuencia es un desfase de 90° para todas las frecuencias positivas.
    # Aquí, usamos el freqz para mostrar la respuesta de un filtro de Hilbert discreto de orden 101.
    from scipy.signal import hilbert, firwin
    fir_hilbert = firwin(101, [0.05, 0.95], pass_zero=False, window="hamming", fs=fs)
    w, h = freqz(fir_hilbert, worN=8000, fs=fs)
    fig4, (ax_mag, ax_phase) = plt.subplots(2, 1, figsize=(7,6))
    ax_mag.plot(w, np.abs(h))
    ax_mag.set_title('Magnitud')
    ax_mag.set_ylabel('|H(f)|')
    ax_mag.set_xlabel('Frecuencia (Hz)')
    ax_mag.grid()
    ax_phase.plot(w, np.angle(h))
    ax_phase.set_title('Fase')
    ax_phase.set_ylabel('Fase (radianes)')
    ax_phase.set_xlabel('Frecuencia (Hz)')
    ax_phase.grid()
    fig4.tight_layout()
    st.pyplot(fig4)

    st.info(
        "Ajusta los parámetros de la señal y observa el desfase característico de 90° introducido por la Transformada de Hilbert. "
        "El diagrama de Bode muestra la respuesta en frecuencia de un filtro de Hilbert discreto."
    )

# ---- SECCIÓN: Modulación QAM ----
elif section == "Modulación QAM":
    st.title("Modulación QAM")
    st.markdown("""
    Implementa un mapeador para 16-QAM, genera las señales I(t) y Q(t), modula sobre una portadora y visualiza la señal QAM resultante, su espectro y el diagrama de constelación.
    """)

    # Parámetros
    n_symbols = st.sidebar.slider("N° de símbolos", 10, 200, 50)
    fc = st.sidebar.slider("Frecuencia de portadora (Hz)", 10, 100, 20)
    fs = st.sidebar.slider("Frecuencia de muestreo (Hz)", 100, 2000, 500)
    np.random.seed(st.sidebar.slider("Semilla aleatoria", 0, 1000, 1))
    symbol_rate = st.sidebar.slider("Símbolos por segundo", 5, 100, 20)

    # Generar símbolos aleatorios (0-15 para 16-QAM)
    symbols = np.random.randint(0, 16, n_symbols)

    # Tabla de mapeo 16-QAM (Gray)
    mapping_table = {
        0: (-3, -3), 1: (-3, -1), 2: (-3, +3), 3: (-3, +1),
        4: (-1, -3), 5: (-1, -1), 6: (-1, +3), 7: (-1, +1),
        8: (+3, -3), 9: (+3, -1), 10: (+3, +3), 11: (+3, +1),
        12: (+1, -3), 13: (+1, -1), 14: (+1, +3), 15: (+1, +1)
    }
    mapped = np.array([mapping_table[s] for s in symbols])
    i_seq = mapped[:, 0]
    q_seq = mapped[:, 1]

    # Tiempo y señales baseband
    samples_per_symbol = int(fs // symbol_rate)
    total_samples = samples_per_symbol * n_symbols
    t = np.arange(total_samples) / fs
    i_signal = np.repeat(i_seq, samples_per_symbol)
    q_signal = np.repeat(q_seq, samples_per_symbol)

    # Señal QAM
    qam_signal = i_signal * np.cos(2 * np.pi * fc * t) - q_signal * np.sin(2 * np.pi * fc * t)

    # Visualización de I(t)*cos y Q(t)*sin → señales suavizadas
    st.subheader("Señales I(t)·cos y Q(t)·sin (componentes suavizadas)")
    i_cos = i_signal * np.cos(2 * np.pi * fc * t)
    q_sin = q_signal * np.sin(2 * np.pi * fc * t)

    fig1, ax1 = plt.subplots()
    ax1.plot(t[:8*samples_per_symbol], i_cos[:8*samples_per_symbol], label='I(t)·cos', color='blue')
    ax1.plot(t[:8*samples_per_symbol], q_sin[:8*samples_per_symbol], label='Q(t)·sin', color='red', linestyle='--')
    ax1.set_xlabel("Tiempo (s)")
    ax1.set_ylabel("Amplitud")
    ax1.legend()
    ax1.grid(True)
    st.pyplot(fig1)


    # Señal QAM en el tiempo (fragmento)
    st.subheader("Señal QAM modulada (fragmento)")
    fig2, ax2 = plt.subplots()
    ax2.plot(t[:8*samples_per_symbol], qam_signal[:8*samples_per_symbol])
    ax2.set_xlabel("Tiempo (s)")
    ax2.set_ylabel("Amplitud")
    st.pyplot(fig2)

    # Espectro de la señal QAM
    st.subheader("Espectro de la señal QAM")
    QAM_FFT = np.fft.fft(qam_signal)
    freqs = np.fft.fftfreq(len(qam_signal), 1/fs)
    fig3, ax3 = plt.subplots()
    ax3.plot(freqs[:len(freqs)//2], np.abs(QAM_FFT[:len(freqs)//2]))
    ax3.set_xlabel("Frecuencia (Hz)")
    ax3.set_ylabel("Magnitud")
    st.pyplot(fig3)

    # Diagrama de Constelación
    st.subheader("Diagrama de constelación (Tx)")
    fig4, ax4 = plt.subplots()
    ax4.scatter(i_seq, q_seq)
    ax4.set_xlabel("I")
    ax4.set_ylabel("Q")
    ax4.set_title("Constelación 16-QAM (símbolos transmitidos)")
    st.pyplot(fig4)

    st.info(
        "Ajusta los parámetros para observar cómo varía la modulación y la constelación de 16-QAM. "
        "La señal QAM resulta de la combinación de las señales I(t) y Q(t) moduladas sobre una portadora."
    )

# ---- SECCIÓN: Canal AWGN + Demodulación + Constelación ----
elif section == "Canal AWGN + Demodulación + Constelación":
    st.title("Canal AWGN + Demodulación + Constelación")
    st.markdown("""
    Simula un canal con ruido (AWGN), implementa un demodulador básico para 16-QAM y visualiza la constelación en el receptor.
    """)

    # Controles
    n_symbols = st.sidebar.slider("N° de símbolos", 20, 300, 100)
    SNR_dB = st.sidebar.slider("SNR (dB)", 0, 40, 15)
    fs = st.sidebar.slider("Frecuencia de muestreo (Hz)", 100, 2000, 500)
    symbol_rate = st.sidebar.slider("Símbolos por segundo", 5, 100, 20)
    fc = st.sidebar.slider("Frecuencia de portadora (Hz)", 10, 100, 20)
    np.random.seed(st.sidebar.slider("Semilla aleatoria", 0, 1000, 1))

    # --- Generar símbolos 16-QAM ---
    symbols = np.random.randint(0, 16, n_symbols)
    mapping_table = {
        0: (-3, -3), 1: (-3, -1), 2: (-3, +3), 3: (-3, +1),
        4: (-1, -3), 5: (-1, -1), 6: (-1, +3), 7: (-1, +1),
        8: (+3, -3), 9: (+3, -1), 10: (+3, +3), 11: (+3, +1),
        12: (+1, -3), 13: (+1, -1), 14: (+1, +3), 15: (+1, +1)
    }
    mapped = np.array([mapping_table[s] for s in symbols])
    i_seq = mapped[:, 0]
    q_seq = mapped[:, 1]

    # --- Señales I/Q (baseband) ---
    samples_per_symbol = int(fs // symbol_rate)
    total_samples = samples_per_symbol * n_symbols
    t = np.arange(total_samples) / fs
    i_signal = np.repeat(i_seq, samples_per_symbol)
    q_signal = np.repeat(q_seq, samples_per_symbol)

    # --- Modulación QAM ---
    qam_signal = i_signal * np.cos(2 * np.pi * fc * t) - q_signal * np.sin(2 * np.pi * fc * t)

    # --- Canal AWGN ---
    signal_power = np.mean(qam_signal**2)
    SNR_linear = 10**(SNR_dB / 10)
    noise_power = signal_power / SNR_linear
    noise = np.sqrt(noise_power) * np.random.randn(len(qam_signal))
    rx_signal = qam_signal + noise

    # --- Demodulación coherente ---
    # Mezclamos con la portadora para obtener I/Q (asumimos sincronía perfecta)
    i_rx = rx_signal * np.cos(2 * np.pi * fc * t)
    q_rx = -rx_signal * np.sin(2 * np.pi * fc * t)
    # Filtrado paso bajo (simple: promediar por símbolo)
    i_rx_syms = np.array([np.mean(i_rx[k*samples_per_symbol:(k+1)*samples_per_symbol]) for k in range(n_symbols)])
    q_rx_syms = np.array([np.mean(q_rx[k*samples_per_symbol:(k+1)*samples_per_symbol]) for k in range(n_symbols)])

    # --- Decisión: detección por distancia mínima a la constelación ---
    constellation_points = np.array(list(mapping_table.values()))
    detected_symbols = []
    for i_hat, q_hat in zip(i_rx_syms, q_rx_syms):
        distances = np.sum((constellation_points - np.array([i_hat, q_hat]))**2, axis=1)
        detected_symbols.append(np.argmin(distances))
    detected_symbols = np.array(detected_symbols)
    i_detected = constellation_points[detected_symbols][:, 0]
    q_detected = constellation_points[detected_symbols][:, 1]

    # --- Métrica de desempeño ---
    n_errors = np.sum(detected_symbols != symbols)
    error_rate = n_errors / n_symbols * 100

    # --- Visualizaciones ---
    st.subheader("Constelación transmitida y recibida")
    fig1, ax1 = plt.subplots()
    ax1.scatter(i_seq, q_seq, label="Tx (ideal)", alpha=0.5, marker='o')
    ax1.scatter(i_rx_syms, q_rx_syms, label="Rx (ruido)", alpha=0.5, marker='x')
    ax1.scatter(i_detected, q_detected, label="Demodulados", alpha=0.8, marker='.')
    ax1.set_xlabel("I")
    ax1.set_ylabel("Q")
    ax1.set_title("Diagrama de Constelación (Rx)")
    ax1.legend()
    st.pyplot(fig1)

    st.subheader("Señal recibida (fragmento)")
    fig2, ax2 = plt.subplots()
    ax2.plot(t[:8*samples_per_symbol], rx_signal[:8*samples_per_symbol])
    ax2.set_xlabel("Tiempo (s)")
    ax2.set_ylabel("Amplitud")
    st.pyplot(fig2)

    st.subheader("Errores de símbolo")
    st.write(f"Errores de símbolo: **{n_errors}** de {n_symbols} ({error_rate:.2f}%)")
    st.info(
        "Ajusta la SNR y otros parámetros para observar el efecto del ruido sobre la constelación y la tasa de error de símbolos (SER). "
        "El demodulador implementado es coherente y básico, para propósitos didácticos."
    )



Overwriting dashboard.py


In [11]:
# 🟢 Celda de código 3: Ejecución de la Aplicación y Exposición Pública

import time, re

# 0) Instala cloudflared si no está disponible
!wget -q https://github.com/cloudflare/cloudflared/releases/latest/download/cloudflared-linux-amd64.deb
!dpkg -i cloudflared-linux-amd64.deb

# 1) Cerrar instancias antiguas de Streamlit y Cloudflared
!pkill -f streamlit
!pkill -f cloudflared

# 2) Iniciar la app Streamlit en background (sin mostrar logs)
!streamlit run dashboard.py &> /dev/null &

# 3) Esperar unos segundos para que Streamlit arranque correctamente
time.sleep(3)

# 4) Lanzar el túnel con el binario oficial de Cloudflared
!cloudflared tunnel --url http://localhost:8501 > cloudflared.log 2>&1 &

# 5) Dar tiempo para que Cloudflared genere la URL pública
time.sleep(8)

# 6) Leer el log y extraer la URL
url = None
with open("cloudflared.log") as f:
    for line in f:
        m = re.search(r"https://[a-z0-9-]+\.trycloudflare\.com", line)
        if m:
            url = m.group(0)
            break

# 7) Mostrar el resultado al usuario
if url:
    print("✅ Tu dashboard está disponible aquí:\n", url)
else:
    print("⚠️ No se pudo obtener la URL. Verifica 'cloudflared.log' para más detalles.")


(Reading database ... 126288 files and directories currently installed.)
Preparing to unpack cloudflared-linux-amd64.deb ...
Unpacking cloudflared (2025.7.0) over (2025.7.0) ...
Setting up cloudflared (2025.7.0) ...
Processing triggers for man-db (2.10.2-1) ...
✅ Tu dashboard está disponible aquí:
 https://grad-ecology-roof-bloggers.trycloudflare.com
